# Import and model code

In [ ]:

!pip install mlflow
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
from datetime import date, timedelta, datetime
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from keras import backend as K
import math

!pip install tensorflow_decision_forests==1.2.0
import tensorflow_decision_forests as tfdf
import random
from tqdm import tqdm

#Need >= 4.8.0
!pip install tweepy==4.8.0
import smtplib 
import socket
import tweepy as tw
from tweepy.errors import TooManyRequests
from tweepy.errors import Forbidden, NotFound, TwitterServerError
from tweepy.errors import BadRequest
import copy
import warnings
import time
import ast
!pip install transformers
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, precision_recall_fscore_support

import scipy.stats as st
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
!pip install tweet-preprocessor
import preprocessor as p

import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

#Library needed for selecting the number of topics
!pip install kneebow
from kneebow.rotor import Rotor

#Library and initalization  needed to interpret topic composition
!pip install openai
import openai


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


!python -m spacy download en_core_web_sm

!databricks configure --host https://community.cloud.databricks.com/
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Shared/base_model_thesis")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.13.0
    Uninstalling tweepy-4.13.0:
      Successfully uninstalled tweepy-4.13.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.9 MB/s eta 0:00:00
Username: ohauglend@gmail.com
Password: 
Repeat for confirmation: 


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2976583595097642', creation_time=1675704450858, experiment_id='2976583595097642', last_update_time=1681327508835, lifecycle_stage='active', name='/Shared/base_model_thesis', tags={'mlflow.experiment.sourceName': '/Shared/base_model_thesis',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'ohauglend@gmail.com',
 'mlflow.ownerId': '1309670178152885'}>

In [ ]:


class Custom_keras_RandomForestModel(tfdf.keras.RandomForestModel):
    
    def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.input_consumer_key = tf.Variable("")
      self.input_consumer_secret = tf.Variable("")
      self.input_access_token = tf.Variable("")
      self.input_access_token_secret = tf.Variable("")
      self.input_bearer_token = tf.Variable("")
    
    @classmethod
    def set_Twitter_API_keys(self,
                             input_consumer_key,
                             input_consumer_secret,
                             input_access_token,
                             input_access_token_secret,
                             input_bearer_token
                             ):
      self.input_consumer_key = input_consumer_key
      self.input_consumer_secret = input_consumer_secret
      self.input_access_token = input_access_token
      self.input_access_token_secret = input_access_token_secret
      self.input_bearer_token = input_bearer_token

    @classmethod
    def get_Twitter_API_keys(self):
      return {'input_consumer_key': self.input_consumer_key,
              'input_consumer_secret': self.input_consumer_secret,
              'input_access_token': self.input_access_token,
              'input_access_token_secret': self.input_access_token_secret,
              'input_bearer_token': self.input_bearer_token}

    @classmethod
    def extract_twitter_data_from_users(self,
                                        input_df_path,
                                        output_folder_path,
                                        RUN_ID,
                                        generate_embeddings_file = True,
                                        local_repository = '/content/',
                                        storing_intervall = 10000,
                                        filter_retweets = False,
                                        filter_replies = False,
                                        conversation_id = None,
                                        only_english = True,
                                        #requires prior import of datetime.datetime
                                        date_since = (datetime.now() - timedelta(days = 7)),
                                        date_until = (datetime.now() - timedelta(days = 0)),
                                        search_tweets = True,
                                        search_30_day = False,
                                        search_full_archive = False,
                                        search_tweets_result_type = 'recent',
                                        search_tweets_tweet_mode = 'extended',
                                        search_30_day_label = 'sandbox',
                                        search_full_archive_label = 'sandbox',
                                        search_tweets_total_queries = 1000000,
                                        search_30_day_number_of_days = 30,
                                        search_30_day_maxResults = 100,
                                        search_full_archive_number_of_days = 20,
                                        search_full_archive_maxResults = 100,
                                        system_change_input_dynamically = False
                                        ):
        
      """
      Function to be used for data extraction if there is a known list of users
      Input schema (input_df_path) should follow that of '/content/drive/MyDrive/Bot detection/Data/Input_to_servers/Getting_tweet_level_features/Getting_tweet_level_features_split_172_18_0_84.json'
      The function has no return
      
      The intention of the function is to provide large scale real world data extraction to be used for model testing and later retrainging/tuning.
      The data collection schema follows that of previous scripts in this project and can largely be divided into three bolks:
        - User meta data
        - Features derived from Yang.et al 
        - Tweet level features

      The function has no return, but stores the extracted data in the specified output_folder_path.
      
      The function takes in input_df_path, output_folder_path, and query as only default arguments.

        input_df_path is used to gather colums/features that the system should extract from Twitter.
          - It is intended to be passed a df that contains user meta data features and features from Yang et al, 
            however, the system should also be able to handle a df that also contains Tweet based features.
        
        output_folder_path is the folder where the data should be stored. 
          - Specify without the final /
          - Stores entries at every storing_intervall iteration
        
        query is the keyword/words used to search Twitter for, for syntax please consult:
        - https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md
        
        Twitter API credentials are set by: - it is assumed that the user will use academic access credentials.
          - input_consumer_key
          - input_consumer_secret
          - input_access_token
          - input_access_token_secret
          - input_bearer_token

      The function takes in several non-default arguments;
        
        storing_intervall controls the frequency of storage, it is defaulted to store every 10 000 entries, later iterations also contian data from previous iterations.
        
        Query manipulation is done by:
          - filter_retweets - If set to True, retweets are removed from the data collected - Defaulted to False
          - filter_replies - If set to True, replies are removed from the data collected - Defaulted to False
          - conversation_id - If an id is specified the system will retrieve replies and retweets assosiated with this conversation_id - Defaulted to None
          - only_english skips all Tweets that are not in english - Defaulted to True:
            - Keep in mind that the base model is only trained on english data
        
        Date range parameters are controled by:
          - date_since - Controls the lower limit of date range taken into account by the system - Defaulted to 7 days before the current day
          - date_until - Controls the upper limit of date range taken into account by the system - Defaulted to 0 days before the current day
        The system is intended to collect all data from the specified days, meaning a high number of queries should be specified
        Date parameters should be passed in as (datetime.now() - timedelta(days = X) - timedelta(hours = Y))
        Keep in mind that cursor works badly with hours, meaning the reccomended approach is to set limit parameters high enough that it will collect the whole date range

        The remaining parameters controls the collection method used to obtain Twitter data.
        - The defaulted one is to use search_tweets as this requires no development environment setup other than normal app credentials. 
          * This is done by setting search_tweets = True, search_30_day = False, and search_full_archive = False
          * search_tweets_result_type controls which Tweets the system prioritizes. It is defaulted to 'recent', possible other values are 'popular' and 'mixed'
          * search_tweets_tweet_mode controls how much of the Tweet is returned. It is defaulted to 'extended'
          * search_tweets_total_queries controls the total number of queries made to Twitter API. This should be set in conjunction with date parameters and is defaulted to a high 1 000 000. 
          
          search_tweets is the defaulted collection method as it is the most feasible approach without purchasing aditional subscriptions.
          Further, it is hypothesised that the most recent data will be the most relevant ones. 
          Hence, it is recommended to use search_tweets, however if needed the system also supports search_30_day and search_full_archive collection methods.
          Other methods require extra development environments, in order to set up see: https://developer.twitter.com/en/account/environments
          Keep in mind that max rate limits per call for sandbox environments are currently 500. 
          If this function is to be used with older data as it can be with search_tweets one therefore, needs to purchase additional subscriptions.

        - search_30_day enables the user to search the 30 day archive of Twitter instead of the 7 day defaulted one. 
          * It requires an environment to be set up within the app, and hence also a label to passed in instantiation, this is controled by the search_30_day_label parameter
          * The function loops through dates in backwards order until search_30_day_number_of_days is met, extracting search_30_day_maxResults per day
            * Increasing this to > 30 has no effect -> will effectively be 30
          * As other (cheaper) methods can handle more recent data, it is recommended to set: date_since = (datetime.now() - timedelta(days = 8)), date_until = (datetime.now() - timedelta(days = 7))
          * search_30_day_maxResults controls the number of queries made to the system - possible values: [10,100]
          If these cannot be handled as missing values one needs to look up the user directly

        - search_full_archive enables the user to search the full archive of Twitter instead of the 7 day defaulted one. 
          * It requires an environment to be set up within the app, and hence also a label to passed in instantiation, this is controled by the search_full_archive_label parameter
          * The function loops through dates in backwards order until search_full_archive_number_of_days is met, extracting search_full_archive_maxResults per day
          * As other (cheaper) methods can handle more recent data, it is recommended to set: date_since = (datetime.now() - timedelta(days = 31)), date_until = (datetime.now() - timedelta(days = 30))
          * search_full_archive_maxResults controls the number of queries made to the system - possible values: [10,100]
          If these cannot be handled as missing values one needs to look up the user directly

        - system_change_input_dynamically enables the system to change between search_tweets, search_30_day, and search_full_archive dynamically based on inputted date parameters.
          It is defaulted to False.



      #imports
      import pandas as pd
      import re
      import numpy as np
      import os
      from tqdm import tqdm


      #Need >= 4.8.0
      !pip install tweepy==4.8.0

      #Setting up email notification for when errors are triggered
      import smtplib 
      import socket

      import tweepy as tw
      from tweepy.errors import TooManyRequests
      from tweepy.errors import Forbidden, NotFound, TwitterServerError
      from tweepy.errors import BadRequest
      #NotFound: 404 Not Found

      import copy
      from datetime import date, timedelta, datetime
      import warnings
      import time

      """


      #Validating that set_Twitter_API_keys has been called
      #input_access_token_secret is not checked as it can be empty (if only essential access)
      if self.input_consumer_key == "" or self.input_consumer_secret == "" or self.input_access_token == "" or self.input_bearer_token == "":
        raise ValueError("Twitter API credentials are empty strings please use model method set_Twitter_API_keys")
      
      else:
        input_consumer_key= self.input_consumer_key
        input_consumer_secret= self.input_consumer_secret
        input_access_token= self.input_access_token
        input_access_token_secret= self.input_access_token_secret
        input_bearer_token= self.input_bearer_token

      #Connecting to the api, setting the keys
      
      try:
        auth = tw.OAuthHandler(input_consumer_key, input_consumer_secret)
        if(input_access_token_secret!= ""):
          auth.set_access_token(input_access_token, input_access_token_secret)
        api = tw.API(auth, wait_on_rate_limit=True)
      except:
        pass
      
      count = 0

      ##
      input_df = pd.read_json(input_df_path)
      input_df = input_df.reset_index(drop = True)
      df = input_df



      entry_lst = []

      
      #Input validation
      if date_until > datetime.now():
        raise ValueError("date_until is after the present date")
        
      if date_since > date_until:
        raise ValueError("date_since is after date_until, the specified start range is after the specified end range")

      if sum([search_tweets, search_30_day, search_full_archive]) >= 2:
        raise ValueError("The Boolean parameters search_tweets, search_30_day, search_full_archive are used to select between different cursors, please only specify one, for usecase see docstring")
      
      #As we would like a lean approach - search_full_archive is more expensive than search_30_day which is more expensive than search_tweets,
      #The system can change implementation based on input feasibility, a warning is also thrown, it can be toggled by the system_change_input_dynamically parameter 
      if system_change_input_dynamically:
        if (datetime.now() - date_since).days <= 7 and (datetime.now() - date_until).days <= 7:
          search_tweets = True
          search_30_day = False
          search_full_archive = False
          warnings.warn('System changed implementation to serch_tweets based on inputted date range\nThis can be toggled of by setting system_change_input_dynamically to False')

        elif (datetime.now() - date_since).days <= 30 and (datetime.now() - date_until).days <= 30:
          search_tweets = False
          search_30_day = True
          search_full_archive = False
          warnings.warn('System changed implementation to search_30_day based on inputted date range\nThis can be toggled of by setting system_change_input_dynamically to False')
        
        else:
          search_tweets = False
          search_30_day = False
          search_full_archive = True
          warnings.warn('System changed implementation to search_full_archive based on inputted date range\nThis can be toggled of by setting system_change_input_dynamically to False')

      
      #looping thorugh columns and clearing content in df except for do_not_clear_columns_lst 
      do_not_clear_columns_lst = ['user_screen_name', 'Bot', 'Source/label']
      for col in df.columns.tolist():
        if col in do_not_clear_columns_lst:
          continue
        if type(df[col].iloc[0]) == np.float64:
          df[col] = [np.nan for _ in range(len(df))]
        else:
          df[col] = ['' for _ in range(len(df))]

      #Specifying tweet specific variables
      df['tweet_id'] = [np.nan for _ in range(len(df))]
      df['tweet_created_at'] = [np.nan for _ in range(len(df))]
      df['tweet_text'] = ['' for _ in range(len(df))]
      df['length_of_text'] = [np.nan for _ in range(len(df))]
      df['is_retweet'] = [np.nan  for _ in range(len(df))]
      df['is_reply'] = [np.nan  for _ in range(len(df))]
      df['tweet_mentions_user_id'] = [np.nan for _ in range(len(df))]
      df['tweet_mentions_user_screen_name'] = [np.nan for _ in range(len(df))]
      df['quote_count'] = [np.nan for _ in range(len(df))]
      df['number_of_tweet_mentions'] = [np.nan for _ in range(len(df))]
      df['retweet_count'] = [np.nan for _ in range(len(df))]
      df['is_retweeted'] = [np.nan for _ in range(len(df))]
      df['favorite_count'] = [np.nan for _ in range(len(df))]
      df['contains_media'] = [np.nan for _ in range(len(df))]
      df['media_ids'] = [np.nan for _ in range(len(df))]
      df['media_types'] = [np.nan for _ in range(len(df))]
      df['media_source_status_ids'] = [np.nan for _ in range(len(df))]
      df['media_original_source_status_id'] = [np.nan for _ in range(len(df))]
      df['media_source_user_ids'] = [np.nan for _ in range(len(df))]
      df['media_original_source_user_id'] = [np.nan for _ in range(len(df))]
      df['source'] = ['' for _ in range(len(df))]

      #Too few entries to be used as features, but can be used for later network analysis
      df['retweeted_status_id'] = [np.nan for _ in range(len(df))]
      df['retweeted_status_user_id'] = [np.nan for _ in range(len(df))]
      df['retweeted_status_user_screen_name'] = ['' for _ in range(len(df))]
      df['in_reply_to_user_id'] = [np.nan for _ in range(len(df))]
      df['in_reply_to_status_id'] = [np.nan for _ in range(len(df))]

      #Setting column types that can hold lists for relevant columns
      for col in ['tweet_mentions_user_id',
                  'tweet_mentions_user_screen_name',
                  'media_ids',
                  'media_types',
                  'media_source_status_ids',
                  'media_source_user_ids',
                  'media_original_source_status_id',
                  'media_original_source_user_id']:
              df[col] = df[col].astype(object)

      #Wrapping it in a try loop so features can be derived from what is collected in case there is an error
      for user_screen_name in tqdm(df.user_screen_name.tolist()):
        query = 'from:'+user_screen_name
        #Defining collection method based on input
        try:
          if search_tweets:
            for entry in tw.Cursor(api.search_tweets,
                                          q=query,
                                          since=date_since.strftime('%Y-%m-%d'),
                                          until=date_until.strftime('%Y-%m-%d'),
                                          result_type  = search_tweets_result_type,
                                          tweet_mode=search_tweets_tweet_mode).items(search_tweets_total_queries):


              #Storring entry_lst at storing_intervall
              if len(entry_lst) % storing_intervall == 0 and len(entry_lst) != 0:
                pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                print("File stored at: "+output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                    
              #As cursor only allows for one look, we loop through it and add entries to entry_lst
              entry_lst.append(entry)
                    
              #Having the system sleep between each entry to limit the load on Twitter API
              time.sleep(0.5)

                    

          elif search_30_day:

            for i in range(search_30_day_number_of_days): 

              try:
                for entry in api.search_30_day(query = query, 
                                          label = search_30_day_label,
                                          fromDate = (date_since -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                          toDate = (date_until -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                          maxResults = search_30_day_maxResults
                                          ):

                  #Storring entry_lst at storing_intervall
                  if len(entry_lst) % storing_intervall == 0 and len(entry_lst) != 0:
                    pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                    print("File stored at: "+output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                        
                  #As cursor only allows for one look, we loop through it and add entries to entry_lst
                  entry_lst.append(entry)
                        
                  #Having the system sleep between each entry to limit the load on Twitter API
                  time.sleep(0.5)
              except:
                  pass


          elif search_full_archive:

            for i in range(search_full_archive_number_of_days):
              
              try:
                
                for entry in api.search_full_archive(query = query, 
                                          label = search_full_archive_label,
                                          fromDate = (date_since -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                          toDate = (date_until -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                          maxResults = search_full_archive_maxResults
                                          ):
                  
                  #Storring entry_lst at storing_intervall
                  if len(entry_lst) % storing_intervall == 0 and len(entry_lst) != 0:
                    pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                    print("File stored at: "+output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                        
                  #As cursor only allows for one look, we loop through it and add entries to entry_lst
                  entry_lst.append(entry)
                        
                  #Having the system sleep between each entry to limit the load on Twitter API
                  time.sleep(0.5)
            
              except:
                pass

                        
      
        except Exception as e:

            #Storring entry_lst
            pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
            print("File stored at: "+output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")

            #Sending oha.digi@cbs.dk an email if exception is triggered
            #Getting local IP
            s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            s.connect(("8.8.8.8", 80))
            LOCAL_IP = s.getsockname()[0]
            s.close()

            #Setting up connection to my gmail
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as connection:  
                email_address = 'ohauglend@gmail.com'
                email_password = 'xoughywacvkfmkhi'
                connection.login(email_address, email_password )
                connection.sendmail(from_addr=email_address, to_addrs='oha.digi@cbs.dk',
                                    msg="IP: "+str(LOCAL_IP)+"\nextract_twitter_data Cursor failed, Error: \t"+ str(e) + '\nNumber of entries in Cursor:\t' +str(len(entry_lst)))
            pass


      #Storing entry_lst in case something happens in populating df
      pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst.json")
      print('entry_lst stored at:\t'+str(output_folder_path)+"/entry_lst.json")


      
      #looping through entry_lst to populate input_df
      for i in tqdm(range(len(entry_lst))):


            #Storing input_df in output_folder_path as a .json file with the user_id of the last user to have features populated
            #Not storing on the first iteration of input_df
            if i % storing_intervall == 0 and i != 0:
                input_df.to_json(output_folder_path+"/last_user_id_"+str(input_df.user_id.iloc[i-1])+".json")
                print("File stored at: "+output_folder_path+"/last_user_id_"+str(input_df.user_id.iloc[i-1])+".json")
        
            #Adding user specific features to the input_df
            #When storing input_df to json the date will be casted to the correct format
            input_df.at[i, 'created_at_1'] = entry_lst[i].user.created_at
            input_df.at[i, 'user_id'] = entry_lst[i].user.id
            input_df.at[i, 'user_name'] = entry_lst[i].user.name
            input_df.at[i, 'user_screen_name'] = entry_lst[i].user.screen_name
            input_df.at[i,'user_location'] = entry_lst[i].user.location

            #user_description 
            if type(entry_lst[i].user.description) == str:
                input_df.at[i, 'user_description'] = entry_lst[i].user.description


            #user_url 
            if type(entry_lst[i].user.url) == str:
                input_df.at[i,'user_url'] = entry_lst[i].user.url

        
            #protected 
            if type(entry_lst[i].user.protected) == bool:
                input_df.at[i,'protected'] = float(entry_lst[i].user.protected)


            #followers_count 
            if type(entry_lst[i].user.followers_count) == int:
                input_df.at[i,'followers_count'] = float(entry_lst[i].user.followers_count)

        
            #friends_count 
            if type(entry_lst[i].user.friends_count) == int:
                input_df.at[i,'friends_count'] = float(entry_lst[i].user.friends_count)


            #listed_count 
            if type(entry_lst[i].user.listed_count) == int:
                input_df.at[i,'listed_count'] = float(entry_lst[i].user.listed_count)



            #favourites_count 
            if type(entry_lst[i].user.favourites_count) == int:
                input_df.at[i,'favourites_count'] = float(entry_lst[i].user.favourites_count)


            #utc_offset 
            #missing values are marked as np.nan meaning comparing on type (float) does not work
            if type(entry_lst[i].user.utc_offset) == str:
                input_df.at[i,'utc_offset'] = entry_lst[i].user.utc_offset


            #time_zone 
            if type(entry_lst[i].user.time_zone) == str:
                input_df.at[i,'time_zone'] = entry_lst[i].user.time_zone


            #geo_enabled 
            if type(entry_lst[i].user.geo_enabled) == bool:
                input_df.at[i,'geo_enabled'] = float(entry_lst[i].user.geo_enabled)


            #verified - might change - for now: all acounts except 3 in dataset have binary labels if verified or not :
            if type(entry_lst[i].user.verified) == bool:
                input_df.at[i,'verified'] = float(entry_lst[i].user.verified)

          
            #statuses_count -update if new
            if type(entry_lst[i].user.statuses_count) == int:
                input_df.at[i,'statuses_count'] = float(entry_lst[i].user.statuses_count)


            #lang 
            if type(entry_lst[i].user.lang) == str:
                input_df.at[i,'lang'] = entry_lst[i].user.lang


            #contributors_enabled 
            if type(entry_lst[i].user.contributors_enabled) == bool:
                input_df.at[i,'contributors_enabled'] = float(entry_lst[i].user.contributors_enabled)


            #is_translator 
            if type(entry_lst[i].user.is_translator) == bool:
                input_df.at[i,'is_translator'] = float(entry_lst[i].user.is_translator)


            #profile_background_color 
            if type(entry_lst[i].user.profile_background_color) == str:
                input_df.at[i,'profile_background_color'] = entry_lst[i].user.profile_background_color


            #profile_background_image_url 
            if type(entry_lst[i].user.profile_background_image_url) == str:
                input_df.at[i,'profile_background_image_url'] = entry_lst[i].user.profile_background_image_url


            #profile_background_image_url_https 
            if type(entry_lst[i].user.profile_background_image_url_https) == str:
                input_df.at[i,'profile_background_image_url_https'] = entry_lst[i].user.profile_background_image_url_https


            #profile_background_tile 
            if type(entry_lst[i].user.profile_background_tile) == bool:
                input_df.at[i,'profile_background_tile'] = float(entry_lst[i].user.profile_background_tile)


            #profile_image_url 
            if type(entry_lst[i].user.profile_image_url) == str:
                input_df.at[i,'profile_image_url'] = entry_lst[i].user.profile_image_url
          

            #profile_image_url_https 
            if type(entry_lst[i].user.profile_image_url_https) == str:
                input_df.at[i,'profile_image_url_https'] = entry_lst[i].user.profile_image_url_https


            #profile_link_color 
            if type(entry_lst[i].user.profile_link_color) == str:
                input_df.at[i,'profile_link_color'] = entry_lst[i].user.profile_link_color


            #profile_sidebar_border_color 
            if type(entry_lst[i].user.profile_sidebar_border_color) == str:
                input_df.at[i,'profile_sidebar_border_color'] = entry_lst[i].user.profile_sidebar_border_color


            #profile_sidebar_fill_color 
            if type(entry_lst[i].user.profile_sidebar_fill_color) == str:
                input_df.at[i,'profile_sidebar_fill_color'] = entry_lst[i].user.profile_sidebar_fill_color


            #profile_text_color 
            if type(entry_lst[i].user.profile_text_color) == str:
                input_df.at[i,'profile_text_color'] = entry_lst[i].user.profile_text_color
          

            #profile_use_background_image 
            if type(entry_lst[i].user.profile_use_background_image) == bool:
                input_df.at[i,'profile_use_background_image'] = float(entry_lst[i].user.profile_use_background_image)


            #translator_type 
            if type(entry_lst[i].user.translator_type) == str:
                input_df.at[i,'translator_type'] = entry_lst[i].user.translator_type


            #withheld 
            if type(entry_lst[i].user.withheld_in_countries) == list:
                input_df.at[i,'withheld'] = entry_lst[i].user.withheld_in_countries

            #default_profile 
            if type(entry_lst[i].user.default_profile) == bool:
                input_df.at[i,'default_profile'] = float(entry_lst[i].user.default_profile)

            #default_profile_image 
            if type(entry_lst[i].user.default_profile_image) == bool:
                input_df.at[i,'default_profile_image'] = float(entry_lst[i].user.default_profile_image)
        


            #Adding the Tweet specific features to input_df
            input_df.at[i, 'tweet_id'] = entry_lst[i].id
            input_df.at[i, 'tweet_created_at'] = entry_lst[i].created_at
        
            if search_tweets:
                input_df.at[i, 'tweet_text'] = entry_lst[i].full_text
                input_df.at[i, 'length_of_text'] = len(entry_lst[i].full_text)

            else:
                input_df.at[i, 'tweet_text'] = entry_lst[i].text
                input_df.at[i, 'length_of_text'] = len(entry_lst[i].text)


        
            try:
                entry_lst[i].retweeted_status
                input_df.at[i, 'is_retweet'] = float(True)
                
                #Although these are too few entries to be used as features, they can be useful for later network-based approaches
                input_df.at[i, 'retweeted_status_id'] = entry_lst[i].retweeted_status.id
                input_df.at[i, 'retweeted_status_user_id'] = entry_lst[i].retweeted_status.user.id
                input_df.at[i, 'retweeted_status_user_screen_name'] = entry_lst[i].retweeted_status.user.screen_name


            except AttributeError:
                input_df.at[i, 'is_retweet'] = float(False)

            try:
                entry_lst[i].in_reply_to_status_id
                input_df.at[i, 'is_reply'] = float(True)
                
                #Although these are too few entries to be used as features, they can be useful for later network-based approaches
                input_df.at[i, 'in_reply_to_status_id'] = entry_lst[i].in_reply_to_status_id
                input_df.at[i, 'in_reply_to_user_id'] = entry_lst[i].in_reply_to_user_id


            except:
                input_df.at[i, 'is_reply'] = float(False)
                
                input_df.at[i, 'tweet_mentions_user_id'] = [entry_lst[i].entities['user_mentions'][_]['id'] for _ in range(len(entry_lst[i].entities['user_mentions']))]
                input_df.at[i, 'tweet_mentions_user_screen_name'] = [entry_lst[i].entities['user_mentions'][_]['screen_name'] for _ in range(len(entry_lst[i].entities['user_mentions']))]
                input_df.at[i, 'number_of_tweet_mentions'] = len(entry_lst[i].entities['user_mentions'])

            try:
                input_df.at[i, 'quote_count'] = entry_lst[i].quote_count
            except AttributeError:
                input_df.at[i, 'quote_count'] = 0

        
            input_df.at[i, 'retweet_count'] = entry_lst[i].retweet_count
            if (entry_lst[i].retweet_count == 0):
                input_df.at[i, 'is_retweeted'] = float(True)
            else:
                input_df.at[i, 'is_retweeted'] = float(False)

        
            try:
                input_df.at[i, 'favorite_count'] = entry_lst[i].favorite_count
            except AttributeError:
                input_df.at[i, 'favorite_count'] = 0

            try:
              entry_lst[i].extended_entities['media']
              input_df.at[i, 'contains_media'] = float(True)
              input_df.at[i, 'media_ids'] = [entry_lst[i].extended_entities['media'][_].get('id') for _ in range(len(entry_lst[i].extended_entities['media']))]
              input_df.at[i, 'media_types'] = [entry_lst[i].extended_entities['media'][_].get('type') for _ in range(len(entry_lst[i].extended_entities['media']))]
              
              if [entry_lst[i].extended_entities['media'][_].get('source_status_id') for _ in range(len(entry_lst[i].extended_entities['media']))] != [None]:
                    input_df.at[i, 'media_source_status_ids'] = [entry_lst[i].extended_entities['media'][_].get('source_status_id') for _ in range(len(entry_lst[i].extended_entities['media']))]
                    input_df.at[i, 'media_original_source_status_id'] = [entry_lst[i].extended_entities['media'][_].get('source_status_id') == entry_lst[i].id for _ in range(len(entry_lst[i].extended_entities['media']))]
                    
              if [entry_lst[i].extended_entities['media'][_].get('source_user_id') for _ in range(len(entry_lst[i].extended_entities['media']))] != [None]:
                    input_df.at[i, 'media_source_user_ids'] = [entry_lst[i].extended_entities['media'][_].get('source_user_id') for _ in range(len(entry_lst[i].extended_entities['media']))]
                    input_df.at[i, 'media_original_source_user_id'] = [entry_lst[i].extended_entities['media'][_].get('source_user_id') == entry_lst[i].user.id for _ in range(len(entry_lst[i].extended_entities['media']))]

        
        
        
            except:
              input_df.at[i, 'contains_media'] = float(False)

        
            input_df.at[i,'source'] = entry_lst[i].source


            #Specifying yang et al features
            try:
                #Getting user age as a variable to compute the ratios - also storing user age in the input_df but it might not be used as a feature
                #The storing of created_at values takes different formats in some cases, either a timestamp or an integer
                try:
                    user_age = int(time.mktime(input_df.tweet_created_at.iloc[i].timetuple())) - int(time.mktime(input_df.created_at_1.iloc[i].timetuple()))
                except ValueError:
                    user_age = int(input_df.tweet_created_at.iloc[i]) - int(input_df.created_at_1.iloc[i])
                input_df.at[i, 'user_age'] = user_age
                input_df.at[i, 'tweet_frequency'] = input_df.statuses_count.iloc[i]/user_age
                input_df.at[i, 'followers_growth_rate'] = input_df.followers_count.iloc[i]/user_age
                input_df.at[i, 'friends_growth_rate'] = input_df.friends_count.iloc[i]/user_age
                input_df.at[i, 'favourites_growth_rate'] = input_df.favourites_count.iloc[i]/user_age
                input_df.at[i, 'listed_growth_rate'] = input_df.listed_count.iloc[i]/user_age
                
                #Poulating features non-dependent on age
                input_df.at[i, 'followers_friends_ratio'] = input_df.followers_count.iloc[i]/input_df.friends_count.iloc[i]
                input_df.at[i, 'screen_name_length'] = len(input_df.user_screen_name.iloc[i])
                input_df.at[i, 'num_digits_in_screen_name'] = sum(c.isdigit() for c in input_df.user_screen_name.iloc[i])
                input_df.at[i, 'name_length'] = len(input_df.user_name.iloc[i])
                input_df.at[i, 'num_digits_in_name'] = sum(c.isdigit() for c in input_df.user_name.iloc[i])
                input_df.at[i, 'description_length'] = len(input_df.user_description.iloc[i])
        
            except TypeError as e:
                print("Exception in Yang et al features, triggered by user:\t"+str(input_df.user_id.iloc[i])+"\nerror:\t"+str(e))





      #Storing one final copy after exiting the for loop
      input_df.to_json(output_folder_path+"/FINAL_last_user_id_"+str(input_df.user_id.iloc[i])+".json")
      print("Twitter data user search file stored at: "+output_folder_path+"/FINAL_last_user_id_"+str(input_df.user_id.iloc[i])+".json")

      data_path = output_folder_path+"/FINAL_last_user_id_"+str(input_df.user_id.iloc[i])+".json"

      #Generating embeddings file based on RUN_ID, and specified output_folder_path, using path to FINAL_last_user_id file as input_df_path
      if generate_embeddings_file:
        embeddings_path = self.generate_embeddings(RUN_ID = RUN_ID,
                                 input_df_path = output_folder_path+"/FINAL_last_user_id_"+str(input_df.user_id.iloc[i])+".json",
                                 output_folder_path = output_folder_path,
                                 local_repository = local_repository)
        
        return data_path, embeddings_path


      return data_path
      


    @classmethod
    def extract_twitter_data_keyword(self,
                                     input_mock_df_path,
                                     output_folder_path,
                                     query,
                                     RUN_ID,
                                     generate_embeddings_file = True,
                                     local_repository = '/content/',
                                     storing_intervall = 10000,
                                     filter_retweets = False,
                                     filter_replies = False,
                                     conversation_id = None,
                                     only_english = True,
                                     date_since = (datetime.now() - timedelta(days = 7)),
                                     date_until = (datetime.now() - timedelta(days = 0)),
                                     search_tweets = True,
                                     search_30_day = False,
                                     search_full_archive = False,
                                     search_tweets_result_type = 'recent',
                                     search_tweets_tweet_mode = 'extended',
                                     search_30_day_label = 'sandbox',
                                     search_full_archive_label = 'sandbox',
                                     search_tweets_total_queries = 1000000,
                                     search_30_day_number_of_days = 30,
                                     search_30_day_maxResults = 100,
                                     search_full_archive_number_of_days = 20,
                                     search_full_archive_maxResults = 100,
                                     system_change_input_dynamically = False,
                                     ):
      
  
      """
      This function is intended to be used for searching for domain specific data
      input_mock_df_path should follow the schema of '/content/drive/MyDrive/Bot detection/Data/Input_to_servers/Feature_engineering_not_found_in_output_from_servers_When_working_on_updating_user_ meta/Yang_2020/172_18_0_84_all_data_after_first_crash_of_yang.json'
      The function has no return

      The intention of the function is to provide large scale real world data extraction to be used for model testing and later retrainging/tuning.
      The data collection schema follows that of previous scripts in this project and can largely be divided into three bolks:
        - User meta data
        - Features derived from Yang.et al 
        - Tweet level features

      The function has no return, but stores the extracted data in the specified output_folder_path.
      
      The function takes in input_mock_df_path, output_folder_path, and query as only default arguments.

        input_mock_df_path is used to gather colums/features that the system should extract from Twitter.
          - It is intended to be passed a df that contains user meta data features and features from Yang et al, 
            however, the system should also be able to handle a df that also contains Tweet based features.
        
        output_folder_path is the folder where the data should be stored. 
          - Specify without the final /
          - Stores entries at every storing_intervall iteration
        
        query is the keyword/words used to search Twitter for, for syntax please consult:
        - https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md
        
        Twitter API credentials are set by: - it is assumed that the user will use academic access credentials.
          - input_consumer_key
          - input_consumer_secret
          - input_access_token
          - input_access_token_secret
          - input_bearer_token

      The function takes in several non-default arguments;
        
        storing_intervall controls the frequency of storage, it is defaulted to store every 10 000 entries, later iterations also contian data from previous iterations.
        
        Query manipulation is done by:
          - filter_retweets - If set to True, retweets are removed from the data collected - Defaulted to False
          - filter_replies - If set to True, replies are removed from the data collected - Defaulted to False
          - conversation_id - If an id is specified the system will retrieve replies and retweets assosiated with this conversation_id - Defaulted to None
          - only_english skips all Tweets that are not in english - Defaulted to True:
            - Keep in mind that the base model is only trained on english data
        
        Date range parameters are controled by:
          - date_since - Controls the lower limit of date range taken into account by the system - Defaulted to 7 days before the current day
          - date_until - Controls the upper limit of date range taken into account by the system - Defaulted to 0 days before the current day
        The system is intended to collect all data from the specified days, meaning a high number of queries should be specified
        Date parameters should be passed in as (datetime.now() - timedelta(days = X) - timedelta(hours = Y))
        Keep in mind that cursor works badly with hours, meaning the reccomended approach is to set limit parameters high enough that it will collect the whole date range

        The remaining parameters controls the collection method used to obtain Twitter data.
        - The defaulted one is to use search_tweets as this requires no development environment setup other than normal app credentials. 
          * This is done by setting search_tweets = True, search_30_day = False, and search_full_archive = False
          * search_tweets_result_type controls which Tweets the system prioritizes. It is defaulted to 'recent', possible other values are 'popular' and 'mixed'
          * search_tweets_tweet_mode controls how much of the Tweet is returned. It is defaulted to 'extended'
          * search_tweets_total_queries controls the total number of queries made to Twitter API. This should be set in conjunction with date parameters and is defaulted to a high 1 000 000. 
          
          search_tweets is the defaulted collection method as it is the most feasible approach without purchasing aditional subscriptions.
          Further, it is hypothesised that the most recent data will be the most relevant ones. 
          Hence, it is recommended to use search_tweets, however if needed the system also supports search_30_day and search_full_archive collection methods.
          Other methods require extra development environments, in order to set up see: https://developer.twitter.com/en/account/environments
          Keep in mind that max rate limits per call for sandbox environments are currently 500. 
          If this function is to be used with older data as it can be with search_tweets one therefore, needs to purchase additional subscriptions.

        - search_30_day enables the user to search the 30 day archive of Twitter instead of the 7 day defaulted one. 
          * It requires an environment to be set up within the app, and hence also a label to passed in instantiation, this is controled by the search_30_day_label parameter
          * The function loops through dates in backwards order until search_30_day_number_of_days is met, extracting search_30_day_maxResults per day
            * Increasing this to > 30 has no effect -> will effectively be 30
          * As other (cheaper) methods can handle more recent data, it is recommended to set: date_since = (datetime.now() - timedelta(days = 8)), date_until = (datetime.now() - timedelta(days = 7))
          * search_30_day_maxResults controls the number of queries made to the system - possible values: [10,100]
          If these cannot be handled as missing values one needs to look up the user directly

        - search_full_archive enables the user to search the full archive of Twitter instead of the 7 day defaulted one. 
          * It requires an environment to be set up within the app, and hence also a label to passed in instantiation, this is controled by the search_full_archive_label parameter
          * The function loops through dates in backwards order until search_full_archive_number_of_days is met, extracting search_full_archive_maxResults per day
          * As other (cheaper) methods can handle more recent data, it is recommended to set: date_since = (datetime.now() - timedelta(days = 31)), date_until = (datetime.now() - timedelta(days = 30))
          * search_full_archive_maxResults controls the number of queries made to the system - possible values: [10,100]
          If these cannot be handled as missing values one needs to look up the user directly

        - system_change_input_dynamically enables the system to change between search_tweets, search_30_day, and search_full_archive dynamically based on inputted date parameters.
          It is defaulted to False.




      #imports
      import pandas as pd
      import re
      import numpy as np
      import os
      from tqdm import tqdm


      #Need >= 4.8.0
      !pip install tweepy==4.8.0

      #Setting up email notification for when errors are triggered
      import smtplib 
      import socket

      import tweepy as tw
      from tweepy.errors import TooManyRequests
      from tweepy.errors import Forbidden, NotFound, TwitterServerError
      from tweepy.errors import BadRequest
      #NotFound: 404 Not Found

      import copy
      from datetime import date, timedelta, datetime
      import warnings
      import time

      """

      #Validating that set_Twitter_API_keys has been called
      #input_access_token_secret is not checked as it can be empty (if only essential access)
      if self.input_consumer_key == "" or self.input_consumer_secret == "" or self.input_access_token == "" or self.input_bearer_token == "":
        raise ValueError("Twitter API credentials are empty strings please use model method set_Twitter_API_keys")
      
      else:
        input_consumer_key= self.input_consumer_key
        input_consumer_secret= self.input_consumer_secret
        input_access_token= self.input_access_token
        input_access_token_secret= self.input_access_token_secret
        input_bearer_token= self.input_bearer_token


      #Connecting to the api, setting the keys      
      auth = tw.OAuthHandler(input_consumer_key, input_consumer_secret)
      if(input_access_token_secret!= ""):
        auth.set_access_token(input_access_token, input_access_token_secret)
      api = tw.API(auth, wait_on_rate_limit=True)

      
      count = 0

      df = pd.read_json(input_mock_df_path)


      entry_lst = []

      
      #Input validation
      if date_until > datetime.now():
        raise ValueError("date_until is after the present date")
        
      if date_since > date_until:
        raise ValueError("date_since is after date_until, the specified start range is after the specified end range")

      if sum([search_tweets, search_30_day, search_full_archive]) >= 2:
        raise ValueError("The Boolean parameters search_tweets, search_30_day, search_full_archive are used to select between different cursors, please only specify one, for usecase see docstring")
      
      #As we would like a lean approach - search_full_archive is more expensive than search_30_day which is more expensive than search_tweets,
      #The system can change implementation based on input feasibility, a warning is also thrown, it can be toggled by the system_change_input_dynamically parameter 
      if system_change_input_dynamically:
        if (datetime.now() - date_since).days <= 7 and (datetime.now() - date_until).days <= 7:
          search_tweets = True
          search_30_day = False
          search_full_archive = False
          warnings.warn('System changed implementation to serch_tweets based on inputted date range\nThis can be toggled of by setting system_change_input_dynamically to False')

        elif (datetime.now() - date_since).days <= 30 and (datetime.now() - date_until).days <= 30:
          search_tweets = False
          search_30_day = True
          search_full_archive = False
          warnings.warn('System changed implementation to search_30_day based on inputted date range\nThis can be toggled of by setting system_change_input_dynamically to False')
        
        else:
          search_tweets = False
          search_30_day = False
          search_full_archive = True
          warnings.warn('System changed implementation to search_full_archive based on inputted date range\nThis can be toggled of by setting system_change_input_dynamically to False')


      #Query manipulation
      query = query.lower()
      query = query.replace(' or ',' ') 
      query = query.replace(' and ',' ') 

      #Allowing the user to specify the conversation_id as the system defaults to the first one retrived matching the query
      if conversation_id == None:
      #Removing cases and and/or from input:
        if filter_retweets:
          query += " AND -filter:retweet"

        if filter_replies:
          query += " AND -filter:replies"

      else:
        query='conversation_id:'+ conversation_id

      if only_english:
        query += ' lang:en'
      
      
      #As df (from input) is just needed for the columns we set df to be equal to its first row
      df = df.iloc[0:1]

      #looping thorugh columns and clearing content in df
      for col in df.columns.tolist():
        if type(df[col].iloc[0]) == np.float64:
          df[col] = [np.nan for _ in range(len(df))]
        else:
          df[col] = ['' for _ in range(len(df))]

      #Specifying tweet specific variables
      df['tweet_id'] = [np.nan for _ in range(len(df))]
      df['tweet_created_at'] = [np.nan for _ in range(len(df))]
      df['tweet_text'] = ['' for _ in range(len(df))]
      df['length_of_text'] = [np.nan for _ in range(len(df))]
      df['is_retweet'] = [np.nan  for _ in range(len(df))]
      df['is_reply'] = [np.nan  for _ in range(len(df))]
      df['tweet_mentions_user_id'] = [np.nan for _ in range(len(df))]
      df['tweet_mentions_user_screen_name'] = [np.nan for _ in range(len(df))]
      df['quote_count'] = [np.nan for _ in range(len(df))]
      df['number_of_tweet_mentions'] = [np.nan for _ in range(len(df))]
      df['retweet_count'] = [np.nan for _ in range(len(df))]
      df['is_retweeted'] = [np.nan for _ in range(len(df))]
      df['favorite_count'] = [np.nan for _ in range(len(df))]
      df['contains_media'] = [np.nan for _ in range(len(df))]
      df['media_ids'] = [np.nan for _ in range(len(df))]
      df['media_types'] = [np.nan for _ in range(len(df))]
      df['media_source_status_ids'] = [np.nan for _ in range(len(df))]
      df['media_original_source_status_id'] = [np.nan for _ in range(len(df))]
      df['media_source_user_ids'] = [np.nan for _ in range(len(df))]
      df['media_original_source_user_id'] = [np.nan for _ in range(len(df))]
      df['source'] = ['' for _ in range(len(df))]

      #Too few entries to be used as features, but can be used for later network analysis
      df['retweeted_status_id'] = [np.nan for _ in range(len(df))]
      df['retweeted_status_user_id'] = [np.nan for _ in range(len(df))]
      df['retweeted_status_user_screen_name'] = ['' for _ in range(len(df))]
      df['in_reply_to_user_id'] = [np.nan for _ in range(len(df))]
      df['in_reply_to_status_id'] = [np.nan for _ in range(len(df))]

      #Setting column types that can hold lists for relevant columns
      for col in ['tweet_mentions_user_id',
                  'tweet_mentions_user_screen_name',
                  'media_ids',
                  'media_types',
                  'media_source_status_ids',
                  'media_source_user_ids',
                  'media_original_source_status_id',
                  'media_original_source_user_id']:
        
        df[col] = df[col].astype(object)


      #Wrapping it in a try loop so features can be derived from what is collected in case there is an error
      try:
        #Defining collection method based on input
        if search_tweets:
          for entry in tqdm(tw.Cursor(api.search_tweets,
                                        q=query,
                                        since=date_since.strftime('%Y-%m-%d'),
                                        until=date_until.strftime('%Y-%m-%d'),
                                        result_type  = search_tweets_result_type,
                                        tweet_mode=search_tweets_tweet_mode).items(search_tweets_total_queries)):
            
            #As cursor only allows for one look, we loop through it and add entries to entry_lst
            entry_lst.append(entry)
            
            #Having the system sleep between each entry to limit the load on Twitter API
            time.sleep(0.5)

        elif search_30_day:
          for i in range(search_30_day_number_of_days):
            
            try:
              
              for entry in tqdm(api.search_30_day(query = query, 
                                    label = search_30_day_label,
                                    fromDate = (date_since -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                    toDate = (date_until -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                    maxResults = search_30_day_maxResults
                                    )):
                
                #Storring entry_lst at storing_intervall
                if len(entry_lst) % storing_intervall == 0 and len(entry_lst) != 0:
                  pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                  print("File stored at: "+output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                      

                #As cursor only allows for one look, we loop through it and add entries to entry_lst
                entry_lst.append(entry)
                
                #Having the system sleep between each entry to limit the load on Twitter API
                time.sleep(0.5)
            
            except:
              pass

        
        elif search_full_archive:

          for i in range(search_full_archive_number_of_days):

            try:
              
              for entry in tqdm(api.search_full_archive(query = query, 
                                    label = search_full_archive_label,
                                    fromDate = (date_since -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                    toDate = (date_until -timedelta(days = i)).strftime('%Y%m%d%H%m'),
                                    maxResults = search_full_archive_maxResults
                                    )):
                
                #Storring entry_lst at storing_intervall
                if len(entry_lst) % storing_intervall == 0 and len(entry_lst) != 0:
                  pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                  print("File stored at: "+output_folder_path+"/entry_lst_length_"+str(len(entry_lst))+".json")
                          
                
                #As cursor only allows for one look, we loop through it and add entries to entry_lst
                entry_lst.append(entry)
                
                #Having the system sleep between each entry to limit the load on Twitter API
                time.sleep(0.5)

            except:
              pass    
      
      except Exception as e:

        #Sending oha.digi@cbs.dk an email if exception is triggered
        #Getting local IP
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        LOCAL_IP = s.getsockname()[0]
        s.close()

        #Setting up connection to my gmail
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as connection:  
            email_address = 'ohauglend@gmail.com'
            email_password = 'xoughywacvkfmkhi'
            connection.login(email_address, email_password )
            connection.sendmail(from_addr=email_address, to_addrs='oha.digi@cbs.dk', 
            msg="IP: "+str(LOCAL_IP)+"\nextract_twitter_data Cursor failed, Error: \t"+ str(e) + '\nNumber of entries in Cursor:\t' +str(len(entry_lst))
            )
        pass



      #Storing entry_lst in case something happens in populating df
      pd.Series(entry_lst).to_json(output_folder_path+"/entry_lst.json")

      #looping through entry_lst to populate df
      for i in range(len(entry_lst)):

        #Storing df in output_folder_path as a .json file with the user_id of the last user to have features populated
        #Not storing on the first iteration of df
        if i % storing_intervall == 0 and i != 0:
            df.to_json(output_folder_path+"/last_user_id_"+str(df.user_id.iloc[i-1])+".json")
            print("File stored at: "+output_folder_path+"/last_user_id_"+str(df.user_id.iloc[i-1])+".json")
        
        #Adding user specific features to the df
        #When storing df to json the date will be casted to the correct format
        df.at[i, 'created_at_1'] = entry_lst[i].user.created_at
        df.at[i, 'user_id'] = entry_lst[i].user.id
        df.at[i, 'user_name'] = entry_lst[i].user.name
        df.at[i, 'user_screen_name'] = entry_lst[i].user.screen_name
        df.at[i,'user_location'] = entry_lst[i].user.location

        #user_description 
        if type(entry_lst[i].user.description) == str:
          df.at[i, 'user_description'] = entry_lst[i].user.description


        #user_url 
        if type(entry_lst[i].user.url) == str:
            df.at[i,'user_url'] = entry_lst[i].user.url

        
        #protected 
        if type(entry_lst[i].user.protected) == bool:
            df.at[i,'protected'] = float(entry_lst[i].user.protected)


        #followers_count 
        if type(entry_lst[i].user.followers_count) == int:
          df.at[i,'followers_count'] = float(entry_lst[i].user.followers_count)

        
        #friends_count 
        if type(entry_lst[i].user.friends_count) == int:
          df.at[i,'friends_count'] = float(entry_lst[i].user.friends_count)


        #listed_count 
        if type(entry_lst[i].user.listed_count) == int:
          df.at[i,'listed_count'] = float(entry_lst[i].user.listed_count)



        #favourites_count 
        if type(entry_lst[i].user.favourites_count) == int:
          df.at[i,'favourites_count'] = float(entry_lst[i].user.favourites_count)


        #utc_offset 
        #missing values are marked as np.nan meaning comparing on type (float) does not work
        if type(entry_lst[i].user.utc_offset) == str:
          df.at[i,'utc_offset'] = entry_lst[i].user.utc_offset


        #time_zone 
        if type(entry_lst[i].user.time_zone) == str:
          df.at[i,'time_zone'] = entry_lst[i].user.time_zone


        #geo_enabled 
        if type(entry_lst[i].user.geo_enabled) == bool:
          df.at[i,'geo_enabled'] = float(entry_lst[i].user.geo_enabled)


        #verified - might change - for now: all acounts except 3 in dataset have binary labels if verified or not :
        if type(entry_lst[i].user.verified) == bool:
          df.at[i,'verified'] = float(entry_lst[i].user.verified)

          
        #statuses_count -update if new
        if type(entry_lst[i].user.statuses_count) == int:
          df.at[i,'statuses_count'] = float(entry_lst[i].user.statuses_count)


        #lang 
        if type(entry_lst[i].user.lang) == str:
          df.at[i,'lang'] = entry_lst[i].user.lang


        #contributors_enabled 
        if type(entry_lst[i].user.contributors_enabled) == bool:
          df.at[i,'contributors_enabled'] = float(entry_lst[i].user.contributors_enabled)


        #is_translator 
        if type(entry_lst[i].user.is_translator) == bool:
          df.at[i,'is_translator'] = float(entry_lst[i].user.is_translator)


        #profile_background_color 
        if type(entry_lst[i].user.profile_background_color) == str:
          df.at[i,'profile_background_color'] = entry_lst[i].user.profile_background_color


        #profile_background_image_url 
        if type(entry_lst[i].user.profile_background_image_url) == str:
          df.at[i,'profile_background_image_url'] = entry_lst[i].user.profile_background_image_url


        #profile_background_image_url_https 
        if type(entry_lst[i].user.profile_background_image_url_https) == str:
          df.at[i,'profile_background_image_url_https'] = entry_lst[i].user.profile_background_image_url_https


        #profile_background_tile 
        if type(entry_lst[i].user.profile_background_tile) == bool:
          df.at[i,'profile_background_tile'] = float(entry_lst[i].user.profile_background_tile)


        #profile_image_url 
        if type(entry_lst[i].user.profile_image_url) == str:
          df.at[i,'profile_image_url'] = entry_lst[i].user.profile_image_url
          

        #profile_image_url_https 
        if type(entry_lst[i].user.profile_image_url_https) == str:
          df.at[i,'profile_image_url_https'] = entry_lst[i].user.profile_image_url_https


        #profile_link_color 
        if type(entry_lst[i].user.profile_link_color) == str:
          df.at[i,'profile_link_color'] = entry_lst[i].user.profile_link_color


        #profile_sidebar_border_color 
        if type(entry_lst[i].user.profile_sidebar_border_color) == str:
          df.at[i,'profile_sidebar_border_color'] = entry_lst[i].user.profile_sidebar_border_color


        #profile_sidebar_fill_color 
        if type(entry_lst[i].user.profile_sidebar_fill_color) == str:
          df.at[i,'profile_sidebar_fill_color'] = entry_lst[i].user.profile_sidebar_fill_color


        #profile_text_color 
        if type(entry_lst[i].user.profile_text_color) == str:
          df.at[i,'profile_text_color'] = entry_lst[i].user.profile_text_color
          

        #profile_use_background_image 
        if type(entry_lst[i].user.profile_use_background_image) == bool:
          df.at[i,'profile_use_background_image'] = float(entry_lst[i].user.profile_use_background_image)


        #translator_type 
        if type(entry_lst[i].user.translator_type) == str:
          df.at[i,'translator_type'] = entry_lst[i].user.translator_type


        #withheld 
        if type(entry_lst[i].user.withheld_in_countries) == list:
          df.at[i,'withheld'] = entry_lst[i].user.withheld_in_countries

        #default_profile 
        if type(entry_lst[i].user.default_profile) == bool:
          df.at[i,'default_profile'] = float(entry_lst[i].user.default_profile)

        #default_profile_image 
        if type(entry_lst[i].user.default_profile_image) == bool:
          df.at[i,'default_profile_image'] = float(entry_lst[i].user.default_profile_image)
        


        #Adding the Tweet specific features to df
        df.at[i, 'tweet_id'] = entry_lst[i].id
        df.at[i, 'tweet_created_at'] = entry_lst[i].created_at
        
        if search_tweets:
          df.at[i, 'tweet_text'] = entry_lst[i].full_text
          df.at[i, 'length_of_text'] = len(entry_lst[i].full_text)

        else:
          df.at[i, 'tweet_text'] = entry_lst[i].text
          df.at[i, 'length_of_text'] = len(entry_lst[i].text)


        
        try:
          entry_lst[i].retweeted_status
          df.at[i, 'is_retweet'] = float(True)

          #Although these are too few entries to be used as features, they can be useful for later network-based approaches
          df.at[i, 'retweeted_status_id'] = entry_lst[i].retweeted_status.id
          df.at[i, 'retweeted_status_user_id'] = entry_lst[i].retweeted_status.user.id
          df.at[i, 'retweeted_status_user_screen_name'] = entry_lst[i].retweeted_status.user.screen_name


        except AttributeError:
          df.at[i, 'is_retweet'] = float(False)

        try:
          entry_lst[i].in_reply_to_status_id
          df.at[i, 'is_reply'] = float(True)

          #Although these are too few entries to be used as features, they can be useful for later network-based approaches
          df.at[i, 'in_reply_to_status_id'] = entry_lst[i].in_reply_to_status_id
          df.at[i, 'in_reply_to_user_id'] = entry_lst[i].in_reply_to_user_id


        except:
          df.at[i, 'is_reply'] = float(False)
        
        
        df.at[i, 'tweet_mentions_user_id'] = [entry_lst[i].entities['user_mentions'][_]['id'] for _ in range(len(entry_lst[i].entities['user_mentions']))]
        df.at[i, 'tweet_mentions_user_screen_name'] = [entry_lst[i].entities['user_mentions'][_]['screen_name'] for _ in range(len(entry_lst[i].entities['user_mentions']))]
        df.at[i, 'number_of_tweet_mentions'] = len(entry_lst[i].entities['user_mentions'])

        try:
          df.at[i, 'quote_count'] = entry_lst[i].quote_count
        except AttributeError:
          df.at[i, 'quote_count'] = 0

        
        df.at[i, 'retweet_count'] = entry_lst[i].retweet_count
        if (entry_lst[i].retweet_count == 0):
          df.at[i, 'is_retweeted'] = float(True)
        else:
          df.at[i, 'is_retweeted'] = float(False)

        
        try:
          df.at[i, 'favorite_count'] = entry_lst[i].favorite_count
        except AttributeError:
          df.at[i, 'favorite_count'] = 0

        try:
          entry_lst[i].extended_entities['media']
          df.at[i, 'contains_media'] = float(True)
          df.at[i, 'media_ids'] = [entry_lst[i].extended_entities['media'][_].get('id') for _ in range(len(entry_lst[i].extended_entities['media']))]
          df.at[i, 'media_types'] = [entry_lst[i].extended_entities['media'][_].get('type') for _ in range(len(entry_lst[i].extended_entities['media']))]

          if [entry_lst[i].extended_entities['media'][_].get('source_status_id') for _ in range(len(entry_lst[i].extended_entities['media']))] != [None]:
            df.at[i, 'media_source_status_ids'] = [entry_lst[i].extended_entities['media'][_].get('source_status_id') for _ in range(len(entry_lst[i].extended_entities['media']))]
            df.at[i, 'media_original_source_status_id'] = [entry_lst[i].extended_entities['media'][_].get('source_status_id') == entry_lst[i].id for _ in range(len(entry_lst[i].extended_entities['media']))]

          if [entry_lst[i].extended_entities['media'][_].get('source_user_id') for _ in range(len(entry_lst[i].extended_entities['media']))] != [None]:
            df.at[i, 'media_source_user_ids'] = [entry_lst[i].extended_entities['media'][_].get('source_user_id') for _ in range(len(entry_lst[i].extended_entities['media']))]
            df.at[i, 'media_original_source_user_id'] = [entry_lst[i].extended_entities['media'][_].get('source_user_id') == entry_lst[i].user.id for _ in range(len(entry_lst[i].extended_entities['media']))]

        
        
        
        except AttributeError:
          df.at[i, 'contains_media'] = float(False)

        
        df.at[i,'source'] = entry_lst[i].source


        #Specifying yang et al features
        try:
          #Getting user age as a variable to compute the ratios - also storing user age in the df but it might not be used as a feature
          user_age = int(time.mktime(df.tweet_created_at.iloc[i].timetuple())) - int(time.mktime(df.created_at_1.iloc[i].timetuple()))
          df.at[i, 'user_age'] = user_age
          df.at[i, 'tweet_frequency'] = df.statuses_count.iloc[i]/user_age
          df.at[i, 'followers_growth_rate'] = df.followers_count.iloc[i]/user_age
          df.at[i, 'friends_growth_rate'] = df.friends_count.iloc[i]/user_age
          df.at[i, 'favourites_growth_rate'] = df.favourites_count.iloc[i]/user_age
          df.at[i, 'listed_growth_rate'] = df.listed_count.iloc[i]/user_age
        
          #Poulating features non-dependent on age
          df.at[i, 'followers_friends_ratio'] = df.followers_count.iloc[i]/df.friends_count.iloc[i]
          df.at[i, 'screen_name_length'] = len(df.user_screen_name.iloc[i])
          df.at[i, 'num_digits_in_screen_name'] = sum(c.isdigit() for c in df.user_screen_name.iloc[i])
          df.at[i, 'name_length'] = len(df.user_name.iloc[i])
          df.at[i, 'num_digits_in_name'] = sum(c.isdigit() for c in df.user_name.iloc[i])
          df.at[i, 'description_length'] = len(df.user_description.iloc[i])
        
        except TypeError as e:
          print("Exception in Yang et al features, triggered by user:\t"+str(df.user_id.iloc[i])+"\nerror:\t"+str(e))





      #Storing one final copy after exiting the for loop
      df.to_json(output_folder_path+"/FINAL_last_user_id_"+str(df.user_id.iloc[i])+".json")
      print("Twitter keyword search file stored at: "+output_folder_path+"/FINAL_last_user_id_"+str(df.user_id.iloc[i])+".json")

      data_path = output_folder_path+"/FINAL_last_user_id_"+str(df.user_id.iloc[i])+".json"


      #Generating embeddings file based on RUN_ID, and specified output_folder_path, using path to FINAL_last_user_id file as input_df_path
      if generate_embeddings_file:
        embeddings_path = self.generate_embeddings(RUN_ID = RUN_ID,
                                 input_df_path = output_folder_path+"/FINAL_last_user_id_"+str(df.user_id.iloc[i])+".json",
                                 output_folder_path = output_folder_path,
                                 local_repository = local_repository)
        
        return data_path, embeddings_path

      
      return data_path




    @classmethod
    def generate_embeddings(self,
                            RUN_ID,
                            input_df_path,
                            output_folder_path,
                            local_repository = '/content/'):
       
        """
        The function generates embeddings from textual features and returns the filepath where this is stored. 
        The function extracts the features and the embeddign model used from the model's associated RUN_ID

        """

        #Making a copy of the df from input_df_path
        df = pd.read_json(input_df_path)    
        tokenized_df = copy.deepcopy(df)

      
        #loading in feature_lst as the model_feature_lsit.txt artifact assosiated with the passed in RUN_ID
        feature_lst = ast.literal_eval(mlflow.artifacts.load_text('runs:/'+ RUN_ID +'/model_feature_list.txt'))
        tokenized_df[tokenized_df.columns[~tokenized_df.columns.isin(feature_lst)]]

        #isolating numeric columns
        number_columns = df._get_numeric_data().columns.tolist()
        number_columns.extend(['tweet_mentions_user_id', 'tweet_mentions_user_screen_name', 'media_ids', 'media_source_status_ids', 'media_original_source_status_id', 'media_source_user_ids', 'media_original_source_user_id'])

        #Isolating text columns as columns not in number_columns
        text_columns = []
        for col in tokenized_df.columns.tolist():
          if col not in number_columns:
            text_columns.append(col)

        #Downloading tokenizer as the tokenizer artifact assosiated with the passed in RUN_ID
        #Storing artifact in local repository for colab this is '/content/'
        mlflow.artifacts.download_artifacts('runs:/'+ RUN_ID +'/tokenizer', dst_path = local_repository)
        
        tokenizer = AutoTokenizer.from_pretrained(local_repository+'tokenizer')


        #Performing tokenization on text columns - extracting input_ids
        for col in tqdm(text_columns):

          #Encoding missing values as an empty string  
          tokenized_df[col] = tokenized_df[col].map(lambda x: x if type(x) == str else '')

          #extracting and storing input_ids
          tokenized_df[col] = tokenized_df[col].map(lambda x: tokenizer(x)['input_ids'])
        

        #Outputting the tokenized_df to the specified output_folder_path
        tokenized_df.to_json(output_folder_path+'/tokenized_data_last_user_id_'+str(tokenized_df.user_id.iloc[-1])+'.json')
        
        print('Embeddings file stored at: '+output_folder_path+'/tokenized_data_last_user_id_'+str(tokenized_df.user_id.iloc[-1])+'.json')

        return output_folder_path+'/tokenized_data_last_user_id_'+str(tokenized_df.user_id.iloc[-1])+'.json'



    @classmethod
    def isolate_humans_and_bots(self,
                                embeddings_path,
                                data_path,
                                dataset_cut_off = 5):
      
      """
      The function uses self to identify likely humans and likely bots from domain data as the dataset_cut_off percentage of the most extreme bot score.
      The intention is to use model functionality such as extract_twitter_data_keyword to obtain new domain data, which should be then fed into this function.
      The user needs to specify paths to embedded data (for prediction) and raw textual data (for isolation/selectiion)
      dataset_cut_off is defaulted to 5 meaning 5% of the accoutns will be labeled as likely_humans, and 5% as likely_bots
      """

      
      embeddings_data = pd.read_json(embeddings_path)
      data_untokenized = pd.read_json(data_path)

      #Need to create a dummy and populate 2 features with dummy bolean
      embeddings_data['Bot'] = 1
      embeddings_data = embeddings_data.drop(['withheld'], axis = 1)
      embeddings_data['is_translation_enabled'] = 0
      embeddings_data['has_extended_profile'] = 0

      #processing step needed on untokenized data
      data_untokenized['tweet_created_at'].apply(lambda x: np.nan if type(x) == pd._libs.tslibs.nattype.NaTType else int(time.mktime(x.timetuple())))

      predict_ds = generate_batch_ds(df = embeddings_data,
                                  embeddings_encoder_type = tf.int64)

      pred = self.predict(predict_ds)

      bot_tweets = embeddings_data[np.where(np.searchsorted(np.percentile(pred, [100 - dataset_cut_off]), pred) == 1, 1, 100) == [1]][['user_id','tweet_id']]
      human_tweets = embeddings_data[np.where(np.searchsorted(np.percentile(pred, [dataset_cut_off]), pred) == 1, 1, 100) == [1]][['user_id','tweet_id']]

      likely_bots_df = data_untokenized[data_untokenized['tweet_id'].isin(bot_tweets['tweet_id'].tolist())].drop_duplicates('tweet_id').reset_index(drop = True)
      likely_humans_df = data_untokenized[data_untokenized['tweet_id'].isin(human_tweets['tweet_id'].tolist())].drop_duplicates('tweet_id').reset_index(drop = True)

      return likely_bots_df, likely_humans_df


    @classmethod
    def numerical_and_boolean_features(self,
                                      RUN_ID,
                                      likely_bots_df,
                                      likely_humans_df
                                      local_repository ='/content'):
      """
      The function analyses likely_humans and likely_bots for mean values and statistical differences in their features.
      RUN_ID is used to extract features from the databricks server associated with the model.
      More specifically, the student-t test and the ANOVA test are employed to check if the numerical and Boolean features are drawn from the same distribution and have the same population mean.
      The analysis result as a nested dictionary following the schema: {feature_name: {Bot: {min: val, max: val, mean: val}}, {Human: {min: val, max: val, mean: val}}, 
      #{Difference: {mean: val, t-stat: (statistic, pvalue), ANOVA: (statistic, pvalue)} is stored in the local_repository.

      The function has no return

      """

      #Extracting the features associated with the RUN_ID from databricks
      feature_lst = ast.literal_eval(mlflow.artifacts.load_text('runs:/'+ RUN_ID +'/model_feature_list.txt'))

      #Isolating numerical and boolean features
      num_or_boolean_features = list(set(likely_bots_df.select_dtypes(include=np.number).columns.tolist()) & set(feature_lst))

      #Isolating numerical and boolean features
      num_features = []
      boolean_features = []

      #Looping through num_or_boolean_features, appending col to boolean_features if the only values in likely_humans_df or likely_humans_df is 0 and 1
      for col in num_or_boolean_features:

        #Excluding features that only have one or fewer possible values
        if len(likely_humans_df[col].value_counts().index.tolist()) <= 1 and len(likely_bots_df[col].value_counts().index.tolist()) <= 1 :
          continue

        #Two possible values in the features means that it will be treated as a boolean feature
        if len(likely_humans_df[col].dropna().value_counts().index.tolist()) <= 2 or len(likely_bots_df[col].dropna().value_counts().index.tolist()) <= 2:
          boolean_features.append(col)
        else:
          num_features.append(col)

      #Syntax: {feature_name: {Bot: {min: val, max: val, mean: val}}, {Human: {min: val, max: val, mean: val}}, 
      #{Difference: {mean: val, t-stat: (statistic, pvalue), ANOVA: (statistic, pvalue)}
      num_features_dic = {}
      for feature in num_features:
        
        feature_dic ={}


        bot_feature_series = likely_bots_df[feature].dropna()
        feature_dic['Bot'] = {'min': float(np.min(bot_feature_series)), 
                              'max': float(np.max(bot_feature_series)),
                              'mean': float(np.mean(bot_feature_series))
                              }
        
        human_feature_series = likely_humans_df[feature].dropna()
        feature_dic['Human'] = {'min': float(np.min(human_feature_series)), 
                              'max': float(np.max(human_feature_series)),
                              'mean': float(np.mean(human_feature_series))
                              }

        
        feature_dic['Difference'] = {'mean': float(np.mean(bot_feature_series)) - float(np.mean(human_feature_series)), 
                              't-stat': st.ttest_ind(bot_feature_series.array, human_feature_series.array, equal_var = True),
                              'ANOVA': st.f_oneway(bot_feature_series.array, human_feature_series.array)
                              }
        
        num_features_dic[feature] = feature_dic


      #Syntax: {feature_name: {Bot: {mean: val}}, {Human: {mean: val}}, 
      #{Difference: {mean: val, t-stat: (statistic, pvalue), ANOVA: (statistic, pvalue)}
      boolean_features_dic = {}
      for feature in boolean_features:
        
        feature_dic ={}


        bot_feature_series = likely_bots_df[feature].dropna()
        feature_dic['Bot'] = {'mean': float(np.mean(bot_feature_series))
                              }
        
        human_feature_series = likely_humans_df[feature].dropna()
        feature_dic['Human'] = {'mean': float(np.mean(human_feature_series))
                              }

        
        feature_dic['Difference'] = {'mean': float(np.mean(bot_feature_series)) - float(np.mean(human_feature_series)), 
                              't-stat': st.ttest_ind(bot_feature_series.array, human_feature_series.array, equal_var = True),
                              'ANOVA': st.f_oneway(bot_feature_series.array, human_feature_series.array)
                              }
        
        boolean_features_dic[feature] = feature_dic

      #Storing the feature dir in the local directory
      feature_dic = {'num_features': num_features_dic, 'boolean_features': boolean_features_dic}

      with open(local_repository+'/feature_dic', 'w') as fp:
          json.dump(feature_dic, fp)

    @classmethod
    def frequent_hashtags(likely_bots_or_humans,
                          local_repository ='/content'
                          ):
      
      """
      The function analyses frequent hashtags of either likely_bots or likely_humans. 
        The user should input the whole data of either.
      The results are stored in local_repository
      The function has no return
      """


      hash_tag_series = likely_bots_or_humans.tweet_text.progress_apply(lambda x: re.findall(r'(#\w+)', x))
      sorted_hash_tag_series = hash_tag_series.progress_apply(lambda x: pd.Series(x)).stack().value_counts()

      plt.style.use(plt.style.available[11])

      hash_tags = hash_tag_series.apply(lambda x: pd.Series(x)).stack().value_counts().iloc[0:10].index.tolist()
      values = hash_tag_series.apply(lambda x: pd.Series(x)).stack().value_counts().iloc[0:10].values

      fig, ax = plt.subplots(figsize=(8,6))
      plt.bar(hash_tags, 
              values, align='center')
      plt.xticks(hash_tags)
      plt.ylim(np.min(values)* 0.8, np.max(values)* 1.1)
      plt.title('Frequency distribution of Hahstags',fontsize = 15)
      ax.set_xticklabels(hash_tags, rotation = 45, ha="right", fontsize=13)
      ax.set_ylabel('Value')
      ax.yaxis.label.set_fontsize(13)
      plt.show()

      plt.savefig(local_repository+'frequency_distribution_of_hashtags.png')


    @classmethod
    def tweet_text_processing(self,
                              likely_bots_or_humans,
                              local_repository ='/content' 
                              ):
      """
      A helping function needed for processing tweet text.
      The function returns processed tweet text from either likely_bots or likely_humans (the user needs to specify)
      """

      
      #Removing URLs Mentions Reserved words (RT, FAV) Emojis Smileys
      clean_tweet_series = likely_bots_or_humans.tweet_text.apply(lambda x: p.clean(x))

      #removing digits
      clean_tweet_series = clean_tweet_series.apply(lambda x: x.replace('\d+', '').lower())

      #removing punctuations
      clean_tweet_series = pd.Series([re.sub(r'[^\w\s]', '', word) for word in clean_tweet_series if re.sub(r'[^\w\s]', '', word)])

      #Lemmatizing 
      lemmatizer = nltk.stem.WordNetLemmatizer()
      tokenizer = TweetTokenizer()

      clean_tweet_series = clean_tweet_series.apply(lambda x: [(lemmatizer.lemmatize(w)) for w in tokenizer.tokenize((x))])

      #removing stop-words
      stop_words = set(stopwords.words('english'))
      clean_tweet_series = clean_tweet_series.apply(lambda x: [item for item in x if item not in stop_words])


      return clean_tweet_series



    def frequent_words(self,
                      clean_tweet_series, 
                      local_repository ='/content'):
      
      """
      The function analyses the frequent words of the clean_tweet_series inputed for frequent words
      The function stores a bar plot in local_repository and has no return 
      """


      words = clean_tweet_series.apply(lambda x: pd.Series(x)).stack().value_counts().iloc[0:10].index
      values = clean_tweet_series.apply(lambda x: pd.Series(x)).stack().value_counts().iloc[0:10].values

      plt.style.use(plt.style.available[11])
      fig, ax = plt.subplots(figsize=(12,8))
      plt.bar(words, 
              values, align='center')
      
      plt.xticks(words)
      plt.ylim(0, np.max(values)* 1.1)
      plt.title('Frequency distribution of words in bot Tweets',fontsize = 24)
      ax.set_xticklabels(words, rotation = 45, ha="right", fontsize=20)
      ax.set_ylabel('Value')
      ax.yaxis.set_tick_params(labelsize=20)
      ax.yaxis.label.set_fontsize(20)
      plt.show()

      plt.savefig(local_repository+'frequency_distribution_of_words.png')


    @classmethod
    def topic_modelling(self,
                        clean_tweet_series,
                        local_repository ='/content',
                        number_of_topics = [2,4,6,8,10],
                        engine = "text-davinci-003",
                        temperature = 0.7,
                        max_tokens = 256,
                        top_p = 1,
                        frequency_penalty= 0,
                        presence_penalty = 0,
                        best_of = 3,
                        openai_key ="OPENAI_KEY"):

      """
      The function performs topic modelling on the inputed clean_tweet_series.
      Using Latent Dirichlet Allocation (LDA), the model framework performs topic modelling to represent Tweets as a combination of latent topics 
      and each of these topics as combinations of different words.

      The model first identifies a corpus in clean_tweet_series before it trains LDA models on different number of topics.
        The number of topics are customizable by the user, but is defaulted to [2,4,6,8,10] to ensure interpretability and sufficient sample size.
      The optimal number of topics is chosen by the model as the average of an elbow analysis of the perplexity curve and a knee analysis of the topic coherence scores
      of the different topics.

      Another LDA model with the optimal number of topics is then trained, its output as the word vector for every topic is then fed to a GPT3 model for topic labelling.
        the egine is defaulted to "text-davinci-003" and the user needs to provide credentials through the openai_key parameter.
        the parameters temperature, max_tokens, top_p, frequency_penalty, presence_penalty, and best_of are OpenAI api controlable parameters that is customizable by the user.

      The model stores the topic labels and word-vectors in local_repository, before it creates and stores a pyLDAvis object as an html file


      """

      
      openai.api_key = openai_key

      # Build the bigram and trigram models
      bigram = gensim.models.Phrases(clean_tweet_series, min_count=5, threshold=100) # higher threshold fewer phrases.
      trigram = gensim.models.Phrases(bigram[clean_tweet_series], threshold=100)


      # Create Dictionary 
      id2word = corpora.Dictionary(clean_tweet_series)  
      
      # Term Document Frequency 
      corpus = [id2word.doc2bow(text) for text in clean_tweet_series]  

      perplexity_log = {}
      coherence_score_log = {}


      for num_topics in number_of_topics:

        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=num_topics, 
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
        

        # Compute Perplexity - a measure of how good the model is. lower the better.
        perplexity_log[str(num_topics)] = lda_model.log_perplexity(corpus)
        
        # Compute Coherence Score - a measure of how good the model is. higher the better.
        coherence_score_log[str(num_topics)] = CoherenceModel(model=lda_model, 
                                                              texts=clean_tweet_series, 
                                                              dictionary=id2word, 
                                                              coherence='c_v').get_coherence()


      plt.figure(figsize=(10, 7))


      sns.lineplot(data=perplexity_log, color="g", label="Perplexity")
      plt.legend(loc=4)
      ax2 = plt.twinx()
      sns.lineplot(data=coherence_score_log, color="b", label="Coherence Score", ax=ax2)

      plt.title("Perplexity and Coherence Scores over number of topics", fontsize=18)
      plt.xlabel("Timestep", fontsize=14)
      plt.ylabel("Score", fontsize=14)
      plt.legend(loc=1)

      plt.show()

      plt.savefig(local_repository+'knee_bow_analysis.png')

      #Identifying the number of topics to be modelled using kneebow library
      #Identifying the elbow in the perplexity_log curve and the knee in the coherence_score_log
      #If conflicted the mean as an integer of the two values are chosen as the number of topics

      #Defining a model to fit the series to
      rotor = Rotor()

      #identifying the elbow in perplexity_log
      rotor.fit_rotate(np.array([[int(key), perplexity_log[key]] for key in perplexity_log.keys()]))
      number_of_topics_perplexity_log = int(list(perplexity_log.keys())[rotor.get_elbow_index()])


      #identifying the knee in coherence_score_log
      rotor.fit_rotate(np.array([[int(key), coherence_score_log[key]] for key in coherence_score_log.keys()]))
      number_of_topics_coherence_score_log = int(list(coherence_score_log.keys())[rotor.get_knee_index()])
      #optional plot of elbow
      #rotor.plot_knee()

      num_topics = int(np.mean([number_of_topics_perplexity_log, number_of_topics_coherence_score_log]))

      lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=num_topics, 
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)


      #Creating a list of topics and the words they compose of
      lst = [(lda_model.print_topics()[_][0] + 1, lda_model.print_topics()[_][1]) for _ in range(len(lda_model.print_topics()))]


      #Adding the task to the promt
      corp = str([(lda_model.print_topics()[_][0] + 1, lda_model.print_topics()[_][1]) for _ in range(len(lda_model.print_topics()))])
      corp += "\n\nWhat are the "+str(num_topics)+" topics about output it as a list in the same order as the input?"

      
          
      #Generating string as the returned tweet from GPT-3
      string =  openai.Completion.create(engine= engine,
                                          prompt= corp, 
                                          temperature= temperature,
                                          max_tokens= max_tokens,
                                          top_p= top_p, 
                                          frequency_penalty= frequency_penalty,
                                          presence_penalty=presence_penalty,
                                          best_of = best_of)["choices"][0].text.strip()

      print('Labeled topics')
      for s in string.split('\n'):
        print(s)

      with open(local_repository+'Topic_labels.txt', 'w') as f:
          for s in string.split('\n'):
            f.write(s)

      # Print the keyword of topics
      print('\nTopic keywords:')
      pprint([(lda_model.print_topics()[_][0] + 1, lda_model.print_topics()[_][1]) for _ in range(len(lda_model.print_topics()))])
      doc_lda = lda_model[corpus]

      with open(local_repository+'Topic_keywords.txt', 'w') as f:
          f.write(str([(lda_model.print_topics()[_][0] + 1, lda_model.print_topics()[_][1]) for _ in range(len(lda_model.print_topics()))]))

      print('\npylda visualisation of topics')
      #Note that the topic numbering is not the same as the previous schema
      pyLDAvis.enable_notebook()
      vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

      #saving as html file
      pyLDAvis.save_html(vis, 'lda.html')


        

# Data processing - making the data ready for the model

## Feature selection and batch mapping function

In [ ]:
df2 = df2[["Bot",
      "user_location",
      "user_description",
      "protected",
      "followers_count",
      "friends_count",
      "listed_count",
      "favourites_count",
      "geo_enabled",
      "statuses_count",
      "contributors_enabled",
      "is_translator",
      "is_translation_enabled",
      "profile_background_color",
      "profile_background_tile",
      "profile_link_color",
      "profile_sidebar_border_color",
      "profile_sidebar_fill_color",
      "profile_text_color",
      "profile_use_background_image",
      "translator_type",
      "has_extended_profile",
      "default_profile",
      "default_profile_image",
      "tweet_frequency",
      "followers_growth_rate",
      "friends_growth_rate",
      "favourites_growth_rate",
      "listed_growth_rate",
      "followers_friends_ratio",
      "screen_name_length",
      "num_digits_in_screen_name",
      "name_length",
      "num_digits_in_name",
      "description_length",
      "media_types",
      "tweet_text",
      "length_of_text",
      "is_retweet",
      "is_reply",
      "number_of_tweet_mentions",
      "quote_count",
      "retweet_count",
      "is_retweeted",
      "favorite_count",
      "contains_media",
      "source"]]



df3 = df3[["Bot",
      "user_location",
      "user_description",
      "protected",
      "followers_count",
      "friends_count",
      "listed_count",
      "favourites_count",
      "geo_enabled",
      "statuses_count",
      "contributors_enabled",
      "is_translator",
      "is_translation_enabled",
      "profile_background_color",
      "profile_background_tile",
      "profile_link_color",
      "profile_sidebar_border_color",
      "profile_sidebar_fill_color",
      "profile_text_color",
      "profile_use_background_image",
      "translator_type",
      "has_extended_profile",
      "default_profile",
      "default_profile_image",
      "tweet_frequency",
      "followers_growth_rate",
      "friends_growth_rate",
      "favourites_growth_rate",
      "listed_growth_rate",
      "followers_friends_ratio",
      "screen_name_length",
      "num_digits_in_screen_name",
      "name_length",
      "num_digits_in_name",
      "description_length",
      "media_types",
      "tweet_text",
      "length_of_text",
      "is_retweet",
      "is_reply",
      "number_of_tweet_mentions",
      "quote_count",
      "retweet_count",
      "is_retweeted",
      "favorite_count",
      "contains_media",
      "source"]]



df3

,Bot,user_location,user_description,protected,followers_count,friends_count,listed_count,favourites_count,geo_enabled,statuses_count,...,length_of_text,is_retweet,is_reply,number_of_tweet_mentions,quote_count,retweet_count,is_retweeted,favorite_count,contains_media,source
0,0,"[101, 102]","[101, 3243, 3497, 1996, 2087, 14953, 15485, 23...",0,3473,753,8,33270,0,4674,...,104,0,1,1,0,0,0,14,0,"[101, 10474, 2005, 25249, 102]"
1,0,"[101, 26947, 4360, 1010, 5978, 102]","[101, 100, 11924, 6958, 3992, 16770, 1024, 101...",0,1578,382,18,1953,0,1159,...,291,0,1,3,0,0,0,0,0,"[101, 10474, 2005, 18059, 102]"
2,0,"[101, 8191, 14129, 2148, 1010, 2634, 102]","[101, 102]",0,210,263,2,14689,0,1107,...,140,1,1,1,0,737,1,0,0,"[101, 10474, 2005, 11924, 102]"
3,0,"[101, 3693, 1996, 3524, 9863, 100, 102]","[101, 1037, 2451, 1997, 9683, 1010, 2199, 1988...",0,85172,3,202,173,0,1096,...,32,0,1,1,0,0,0,3,0,"[101, 10474, 4773, 10439, 102]"
4,0,"[101, 16938, 102]","[101, 102]",0,58,243,0,16659,1,4946,...,140,1,1,2,0,1391,1,0,0,"[101, 10474, 2005, 11924, 102]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,"[101, 102]","[101, 15536, 14416, 21541, 24532, 2102, 1064, ...",0,291,1188,0,686,1,330,...,140,1,1,1,0,1,1,0,0,"[101, 10474, 2005, 11924, 102]"
96,0,"[101, 2267, 2380, 1010, 9108, 102]","[101, 11268, 1999, 1996, 29466, 1997, 8827, 17...",0,896,837,1,3193,1,717,...,144,1,1,2,0,55,1,0,0,"[101, 10474, 2005, 18059, 102]"
97,0,"[101, 102]","[101, 2023, 2003, 1996, 10474, 4070, 1997, 199...",0,5403,5464,57,21116,1,8101,...,140,1,1,1,0,12,1,0,0,"[101, 10474, 2005, 18059, 102]"
98,1,"[101, 2002, 1013, 2032, 102]","[101, 2280, 6951, 2005, 5325, 3793, 2240, 1012...",0,341,1171,1,2083,0,1854,...,144,1,1,1,0,10454,1,0,0,"[101, 10474, 4773, 10439, 102]"


## Generating training and test dataset and using pbs batch function to get batched data set


In [ ]:

def pds(example):
  """
  Function used to isolate features and label for train/test
  """
  label = (example["label"] + 1) // 2
  return {
      "user_location": example["user_location"],
      "user_description": example["user_description"],
      "protected": example["protected"],
      "followers_count": example["followers_count"],
      "friends_count": example["friends_count"],
      "listed_count": example["listed_count"],
      "favourites_count": example["favourites_count"],
      "geo_enabled": example["geo_enabled"],
      "statuses_count": example["statuses_count"],
      "contributors_enabled": example["contributors_enabled"],
      "is_translator": example["is_translator"],
      "is_translation_enabled": example["is_translation_enabled"],
      "profile_background_color": example["profile_background_color"],
      "profile_background_tile": example["profile_background_tile"],
      "profile_link_color": example["profile_link_color"],
      "profile_sidebar_border_color": example["profile_sidebar_border_color"],
      "profile_sidebar_fill_color": example["profile_sidebar_fill_color"],
      "profile_text_color": example["profile_text_color"],
      "profile_use_background_image": example["profile_use_background_image"],
      "translator_type": example["translator_type"],
      "has_extended_profile": example["has_extended_profile"],
      "default_profile": example["default_profile"],
      "default_profile_image": example["default_profile_image"],
      "tweet_frequency": example["tweet_frequency"],
      "followers_growth_rate": example["followers_growth_rate"],
      "friends_growth_rate": example["friends_growth_rate"],
      "favourites_growth_rate": example["favourites_growth_rate"],
      "listed_growth_rate": example["listed_growth_rate"],
      "followers_friends_ratio": example["followers_friends_ratio"],
      "screen_name_length": example["screen_name_length"],
      "num_digits_in_screen_name": example["num_digits_in_screen_name"],
      "name_length": example["name_length"],
      "num_digits_in_name": example["num_digits_in_name"],
      "description_length": example["description_length"],
      "media_types": example["media_types"],
      "tweet_text": example["tweet_text"],
      "length_of_text": example["length_of_text"],
      "is_retweet": example["is_retweet"],
      "is_reply": example["is_reply"],
      "number_of_tweet_mentions": example["number_of_tweet_mentions"],
      "quote_count": example["quote_count"],
      "retweet_count": example["retweet_count"],
      "is_retweeted": example["is_retweeted"],
      "favorite_count": example["favorite_count"],
      "contains_media": example["contains_media"],
      "source": example["source"],
      }, label

In [ ]:
def generate_batch_ds(df,
                      embeddings_encoder_type = tf.int64):
  """
  This function uses the pds function to map data from tabular format into the required tensorflow batched dataset.
  The varaible embeddings_encoder_type controls the cast format of the embedded textual features. 
    It should reflect the embedded features and is defaulted to tf.int64

  """


  label = tf.convert_to_tensor(df["Bot"], dtype = tf.int64)
  #label = tf.ragged.constant(df["Bot"], dtype = tf.int64)


  user_location = tf.ragged.constant(df["user_location"], dtype = embeddings_encoder_type)
  user_description = tf.ragged.constant(df["user_description"], dtype = embeddings_encoder_type)
  protected = tf.ragged.constant(df["protected"], dtype = tf.int64)
  followers_count = tf.ragged.constant(df["followers_count"], dtype = tf.int64)
  friends_count = tf.ragged.constant(df["friends_count"], dtype = tf.int64)
  listed_count = tf.ragged.constant(df["listed_count"], dtype = tf.int64)
  favourites_count = tf.ragged.constant(df["favourites_count"], dtype = tf.int64)
  geo_enabled = tf.ragged.constant(df["geo_enabled"], dtype = tf.int64)
  statuses_count = tf.ragged.constant(df["statuses_count"], dtype = tf.int64)
  contributors_enabled = tf.ragged.constant(df["contributors_enabled"], dtype = tf.int64)
  is_translator = tf.ragged.constant(df["is_translator"], dtype = tf.int64)
  is_translation_enabled = tf.ragged.constant(df["is_translation_enabled"], dtype = tf.int64)
  profile_background_color = tf.ragged.constant(df["profile_background_color"], dtype = embeddings_encoder_type)
  profile_background_tile = tf.ragged.constant(df["profile_background_tile"], dtype = tf.int64)
  profile_link_color = tf.ragged.constant(df["profile_link_color"], dtype = embeddings_encoder_type)
  profile_sidebar_border_color = tf.ragged.constant(df["profile_sidebar_border_color"], dtype = embeddings_encoder_type)
  profile_sidebar_fill_color = tf.ragged.constant(df["profile_sidebar_fill_color"], dtype = embeddings_encoder_type)
  profile_text_color = tf.ragged.constant(df["profile_text_color"], dtype = embeddings_encoder_type)
  profile_use_background_image = tf.ragged.constant(df["profile_use_background_image"], dtype = tf.int64)
  translator_type = tf.ragged.constant(df["translator_type"], dtype = embeddings_encoder_type)
  has_extended_profile = tf.ragged.constant(df["has_extended_profile"], dtype = tf.int64)
  default_profile = tf.ragged.constant(df["default_profile"], dtype = tf.int64)
  default_profile_image = tf.ragged.constant(df["default_profile_image"], dtype = tf.int64)
  tweet_frequency = tf.ragged.constant(df["tweet_frequency"], dtype = tf.int64)
  followers_growth_rate = tf.ragged.constant(df["followers_growth_rate"], dtype = tf.int64)
  friends_growth_rate = tf.ragged.constant(df["friends_growth_rate"], dtype = tf.int64)
  favourites_growth_rate = tf.ragged.constant(df["favourites_growth_rate"], dtype = tf.int64)
  listed_growth_rate = tf.ragged.constant(df["listed_growth_rate"], dtype = tf.int64)
  followers_friends_ratio = tf.ragged.constant(df["followers_friends_ratio"], dtype = tf.int64)
  screen_name_length = tf.ragged.constant(df["screen_name_length"], dtype = tf.int64)
  num_digits_in_screen_name = tf.ragged.constant(df["num_digits_in_screen_name"], dtype = tf.int64)
  name_length = tf.ragged.constant(df["name_length"], dtype = tf.int64)
  num_digits_in_name = tf.ragged.constant(df["num_digits_in_name"], dtype = tf.int64)
  description_length = tf.ragged.constant(df["description_length"], dtype = tf.int64)
  media_types = tf.ragged.constant(df["media_types"], dtype = embeddings_encoder_type)
  tweet_text = tf.ragged.constant(df["tweet_text"], dtype = embeddings_encoder_type)
  length_of_text = tf.ragged.constant(df["length_of_text"], dtype = tf.int64)
  is_retweet = tf.ragged.constant(df["is_retweet"], dtype = tf.int64)
  is_reply = tf.ragged.constant(df["is_reply"], dtype = tf.int64)
  number_of_tweet_mentions = tf.ragged.constant(df["number_of_tweet_mentions"], dtype = tf.int64)
  quote_count = tf.ragged.constant(df["quote_count"], dtype = tf.int64)
  retweet_count = tf.ragged.constant(df["retweet_count"], dtype = tf.int64)
  is_retweeted = tf.ragged.constant(df["is_retweeted"], dtype = tf.int64)
  favorite_count = tf.ragged.constant(df["favorite_count"], dtype = tf.int64)
  contains_media = tf.ragged.constant(df["contains_media"], dtype = tf.int64)
  source = tf.ragged.constant(df["source"], dtype = embeddings_encoder_type)


  #train
  return tf.data.Dataset.from_tensor_slices({'user_location': user_location,
                                              'user_description': user_description,
                                              'protected': protected,
                                              'followers_count': followers_count,
                                              'friends_count': friends_count,
                                              'listed_count': listed_count,
                                              'favourites_count': favourites_count,
                                              'geo_enabled': geo_enabled,
                                              'statuses_count': statuses_count,
                                              'contributors_enabled': contributors_enabled,
                                              'is_translator': is_translator,
                                              'is_translation_enabled': is_translation_enabled,
                                              'profile_background_color': profile_background_color,
                                              'profile_background_tile': profile_background_tile,
                                              'profile_link_color': profile_link_color,
                                              'profile_sidebar_border_color': profile_sidebar_border_color,
                                              'profile_sidebar_fill_color': profile_sidebar_fill_color,
                                              'profile_text_color': profile_text_color,
                                              'profile_use_background_image': profile_use_background_image,
                                              'translator_type': translator_type,
                                              'has_extended_profile': has_extended_profile,
                                              'default_profile': default_profile,
                                              'default_profile_image': default_profile_image,
                                              'tweet_frequency': tweet_frequency,
                                              'followers_growth_rate': followers_growth_rate,
                                              'friends_growth_rate': friends_growth_rate,
                                              'favourites_growth_rate': favourites_growth_rate,
                                              'listed_growth_rate': listed_growth_rate,
                                              'followers_friends_ratio': followers_friends_ratio,
                                              'screen_name_length': screen_name_length,
                                              'num_digits_in_screen_name': num_digits_in_screen_name,
                                              'name_length': name_length,
                                              'num_digits_in_name': num_digits_in_name,
                                              'description_length': description_length,
                                              'media_types': media_types,
                                              'tweet_text': tweet_text,
                                              'length_of_text': length_of_text,
                                              'is_retweet': is_retweet,
                                              'is_reply': is_reply,
                                              'number_of_tweet_mentions': number_of_tweet_mentions,
                                              'quote_count': quote_count,
                                              'retweet_count': retweet_count,
                                              'is_retweeted': is_retweeted,
                                              'favorite_count': favorite_count,
                                              'contains_media': contains_media,
                                              'source': source,
                                              'label': label
                                              #'label': label
                                              }).batch(1000).map(pds)


train_ds = generate_batch_ds(df = df2,
                             embeddings_encoder_type = tf.int64)

test_ds = generate_batch_ds(df = df3,
                             embeddings_encoder_type = tf.int64)

# Model training

In [ ]:
# Specify the model.
m = Custom_keras_RandomForestModel(num_trees = 10,
                                 verbose = 2,
                                 max_depth = 20,
                                 compute_oob_variable_importances = True,
                                 compute_oob_performances = True,
                                 random_seed = 42 #Same as used for train/test split
                                 )





# Train the model.
m.fit(x=train_ds)

Use 4 thread(s) for training
Use /tmp/tmpnu_zdm1q as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'user_location': tf.RaggedTensor(values=Tensor("data_55:0", shape=(None,), dtype=int64), row_splits=Tensor("data_56:0", shape=(None,), dtype=int64)), 'user_description': tf.RaggedTensor(values=Tensor("data_53:0", shape=(None,), dtype=int64), row_splits=Tensor("data_54:0", shape=(None,), dtype=int64)), 'protected': <tf.Tensor 'data_41:0' shape=(None,) dtype=int64>, 'followers_count': <tf.Tensor 'data_8:0' shape=(None,) dtype=int64>, 'friends_count': <tf.Tensor 'data_11:0' shape=(None,) dtype=int64>, 'listed_count': <tf.Tensor 'data_21:0' shape=(None,) dtype=int64>, 'favourites_count': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>, 'geo_enabled': <tf.Tensor 'data_13:0' shape=(None,) dtype=int64>, 'statuses_count': <tf.Tensor 'data_47:0' shape=(None,) dtype=int64>, 'contributors_enabled': <tf.Tensor 'data_1:0' shape=(None,) dtype=int64>, 'is

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Normalized tensor features:
 {'user_location': SemanticTensor(semantic=<Semantic.CATEGORICAL_SET: 4>, tensor=tf.RaggedTensor(values=Tensor("Add:0", shape=(None,), dtype=int32), row_splits=Tensor("data_56:0", shape=(None,), dtype=int64))), 'user_description': SemanticTensor(semantic=<Semantic.CATEGORICAL_SET: 4>, tensor=tf.RaggedTensor(values=Tensor("Add_1:0", shape=(None,), dtype=int32), row_splits=Tensor("data_54:0", shape=(None,), dtype=int64))), 'protected': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_2:0' shape=(None,) dtype=float32>), 'followers_count': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_3:0' shape=(None,) dtype=float32>), 'friends_count': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_4:0' shape=(None,) dtype=float32>), 'listed_count': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_5:0' shape=(None,) dtype=float32>), 'favourites_count': SemanticTensor(semantic=<S

[INFO 2023-05-11T06:42:00.52523958+00:00 kernel.cc:756] Start Yggdrasil model training
[INFO 2023-05-11T06:42:00.525339692+00:00 kernel.cc:757] Collect training examples
[INFO 2023-05-11T06:42:00.525557288+00:00 kernel.cc:388] Number of batches: 1
[INFO 2023-05-11T06:42:00.525566483+00:00 kernel.cc:389] Number of examples: 1000
[INFO 2023-05-11T06:42:00.532145933+00:00 kernel.cc:774] Training dataset:
Number of records: 1000
Number of columns: 47

Number of columns by type:
	NUMERICAL: 35 (74.4681%)
	CATEGORICAL_SET: 11 (23.4043%)
	CATEGORICAL: 1 (2.12766%)

Columns:

NUMERICAL: 35 (74.4681%)
	1: "contains_media" NUMERICAL mean:0.212 min:0 max:1 sd:0.408725
	2: "contributors_enabled" NUMERICAL mean:0 min:0 max:0 sd:0
	3: "default_profile" NUMERICAL mean:0.393 min:0 max:1 sd:0.488417
	4: "default_profile_image" NUMERICAL mean:0 min:0 max:0 sd:0
	5: "description_length" NUMERICAL mean:114.777 min:0 max:190 sd:44.6882
	6: "favorite_count" NUMERICAL mean:343.782 min:0 max:35687 sd:2339.07


Model trained in 0:00:00.884629
Compiling model...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
